In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
topwords50 = ['지역', '산림', '문화', '숲', '사람', '교육', '자연', '마을', '이용', '산', '나무', '관광', '생활', '역사', '마음', '활용', '이야기', '미래', '가을', '예술', '청소년', '전통', '삶', '음식', '여름', '소나무', '봄', '이해', '여성', '겨울', '지식', '공감', '정신', '공동체', '사상', '성인', '공예', '정서', '창조', '고급', '남성', '신앙', '교양', '유람', '유람기', '향유', '의식주', '포용', '금송', '성황당', '서낭당', '친화력']
# '학습'을 제거하고 '교육'으로 통합시킴

In [3]:
conc_df = pd.DataFrame()
for i in range(1,13):
    df = pd.read_csv('산문화/산+문화_2022_%s월_형태소.csv'%i)
    conc_df = pd.concat([conc_df, df])

In [4]:
print(conc_df.shape, conc_df.columns)
docs = conc_df['morphs'].to_list()
print(len(docs))

(93151, 2) Index(['org_idx', 'morphs'], dtype='object')
93151


In [6]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
for i in tqdm(range(len(docs))):
    filtered_lst = []
    for w in docs2[i].split():
        if w in topwords50:
            filtered_lst.append(w)
        elif w == '학습':
            filtered_lst.append('교육')
    else:
        docs2[i] = " ".join(filtered_lst)

  0%|          | 0/93151 [00:00<?, ?it/s]

In [ ]:
# #### 중복 제거 잘 됐나 개수로 확인
# print(len(docs[0]))
# print(len(docs2[0]))
# print(docs2[0][0:10])

In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = docs2.copy()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)
print(tdm.shape)

(93151, 50)


In [9]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.18862327, 0.15245064, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.60206392, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.80313706, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.33790733, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.3298509 , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [10]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.188623,0.152451,0.0,0.078859,0.0,0.0,0.000000,0.0,0.000000,0.0,0.148118,0.0,0.920019,0.023495,0.014922,0.000000,0.000000,0.087696,0.0,0.043081,0.0,0.061742,0.000000,0.0,0.0,0.186802,0.000000,0.0,0.033477,0.000000,0.027587,0.0,0.0,0.032530,0.0,0.052116,0.00000,0.0,0.000000,0.10582,0.0,0.000000,0.000000,0.026567,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.511514,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.427554,0.000000,0.000000,0.000000,0.0,0.205738,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.466048,0.0,0.000000,0.00000,0.0,0.388785,0.00000,0.0,0.000000,0.000000,0.380615,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.602064,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.235715,0.000000,0.000000,0.554134,0.0,0.226851,0.0,0.000000,0.472685,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.176809,0.112292,0.000000,0.000000,0.131991,0.0,0.108069,0.0,0.929278,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.251934,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.066450,0.0,0.000000,0.0,0.000000,0.232619,0.059094,0.226651,0.000000,0.625154,0.0,0.028436,0.0,0.061130,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.082977,0.054626,0.0,0.0,0.000000,0.0,0.670789,0.00000,0.0,0.000000,0.00000,0.0,0.139428,0.080814,0.000000,0.0,0.0,0.0,0.0,0.0,0.073811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93146,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.379123,0.000000,0.190616,0.000000,0.431117,0.000000,0.0,0.366895,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.415555,0.0,0.000000,0.35285,0.0,0.000000,0.00000,0.0,0.449742,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
93147,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.551268,0.000000,0.000000,0.000000,0.000000,0.0,0.336945,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.763264,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
93148,0.000000,0.803137,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.157219,0.000000,0.000000,0.554400,0.0,0.151307,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
93149,0.337907,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.300585,0.0,0.000000,0.0,0.000000,0.000000,0.066828,0.341748,0.000000,0.706963,0.0,0.064315,0.0,0.138259,0.000000,0.0,0.0,0.000000,0.322212,0.0,0.149932,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.157674,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [11]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [12]:
tfidf_DF

,가을,겨울,고급,공감,공동체,공예,관광,교양,교육,금송,나무,남성,마을,마음,문화,미래,봄,사람,사상,산,산림,삶,생활,서낭당,성인,소나무,숲,신앙,여름,여성,역사,예술,유람,음식,의식주,이야기,이용,이해,자연,전통,정서,정신,지식,지역,창조,청소년,친화력,포용,향유,활용
0,0.188623,0.152451,0.0,0.078859,0.0,0.0,0.000000,0.0,0.000000,0.0,0.148118,0.0,0.920019,0.023495,0.014922,0.000000,0.000000,0.087696,0.0,0.043081,0.0,0.061742,0.000000,0.0,0.0,0.186802,0.000000,0.0,0.033477,0.000000,0.027587,0.0,0.0,0.032530,0.0,0.052116,0.00000,0.0,0.000000,0.10582,0.0,0.000000,0.000000,0.026567,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.511514,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.427554,0.000000,0.000000,0.000000,0.0,0.205738,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.466048,0.0,0.000000,0.00000,0.0,0.388785,0.00000,0.0,0.000000,0.000000,0.380615,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.602064,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.235715,0.000000,0.000000,0.554134,0.0,0.226851,0.0,0.000000,0.472685,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.176809,0.112292,0.000000,0.000000,0.131991,0.0,0.108069,0.0,0.929278,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.251934,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.066450,0.0,0.000000,0.0,0.000000,0.232619,0.059094,0.226651,0.000000,0.625154,0.0,0.028436,0.0,0.061130,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.082977,0.054626,0.0,0.0,0.000000,0.0,0.670789,0.00000,0.0,0.000000,0.00000,0.0,0.139428,0.080814,0.000000,0.0,0.0,0.0,0.0,0.0,0.073811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93146,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.379123,0.000000,0.190616,0.000000,0.431117,0.000000,0.0,0.366895,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.415555,0.0,0.000000,0.35285,0.0,0.000000,0.00000,0.0,0.449742,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
93147,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.551268,0.000000,0.000000,0.000000,0.000000,0.0,0.336945,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.763264,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
93148,0.000000,0.803137,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.157219,0.000000,0.000000,0.554400,0.0,0.151307,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
93149,0.337907,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.300585,0.0,0.000000,0.0,0.000000,0.000000,0.066828,0.341748,0.000000,0.706963,0.0,0.064315,0.0,0.138259,0.000000,0.0,0.0,0.000000,0.322212,0.0,0.149932,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.00000,0.0,0.157674,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
# 각 단어에 대한 TF-IDF의 합
word_TFIDF_sum = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_TFIDF_sum = word_TFIDF_sum.sort_values(by='tf-idf', ascending=False).reset_index(drop=True)
word_TFIDF_sum

In [14]:
# 각 단어에 대한 TF-IDF의 합
word_TFIDF_average = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.mean(axis=0).flat
})
word_TFIDF_average = word_TFIDF_average.sort_values(by='tf-idf', ascending=False).reset_index(drop=True)
word_TFIDF_average

,단어,tf-idf
0,사람,0.235942
1,산,0.158465
2,문화,0.145879
3,마음,0.101902
4,마을,0.099163
5,지역,0.095237
6,이야기,0.087497
7,역사,0.078869
8,이용,0.077458
9,자연,0.074075


In [15]:
word_TFIDF_average.to_excel("산문화_tfidf_top50.xlsx", index=False)